In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import  pyplot as plt

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

In [3]:
!pip install ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from IPython.core.pylabtools import figsize

In [ ]:
from google.colab import files
uploaded = files.upload() 

In [ ]:

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [ ]:
tf.random.set_seed(1234)


In [ ]:
df = pd.read_csv('Bangladesh.csv',index_col='datetime_utc',parse_dates=False,error_bad_lines=False)

In [ ]:
df.describe()

In [ ]:
len(df)

In [ ]:
df.describe()

In [ ]:
x = float("nan")
print(f"It's df.isna  : {df.isna()}")

In [ ]:
df = df.apply (pd.to_numeric, errors='coerce')
df = df.dropna()

print (df)

In [ ]:
print(df.isnull().sum())

In [ ]:
print (df)


In [ ]:
df=df.iloc[:2000]

In [ ]:
def timeseries (x_axis, y_axis, x_label, y_label):
    plt.figure(figsize = (10, 6))
    plt.plot(x_axis, y_axis, color ='black')
    plt.xlabel(x_label) 
    plt.ylabel(y_label)
timeseries(df.index, df[' _tempm'],'DayTime','temp')


In [ ]:
train_size = int(len(df)*0.8)
train_dataset, test_dataset = df.iloc[:train_size],df.iloc[train_size:]

In [ ]:

print('Dimension of train data: ',train_dataset.shape)
print('Dimension of test data: ', test_dataset.shape)

In [ ]:
X_train = train_dataset.drop(' _tempm', axis = 1)
y_train = train_dataset.loc[:,[' _tempm']]

X_test = test_dataset.drop(' _tempm', axis = 1)
y_test = test_dataset.loc[:,[' _tempm']]

In [ ]:
scaler_x = MinMaxScaler(feature_range = (0,1))
scaler_y = MinMaxScaler(feature_range = (0,1))
input_scaler = scaler_x.fit(X_train)
output_scaler = scaler_y.fit(y_train)
train_y_norm = output_scaler.transform(y_train)
train_x_norm = input_scaler.transform(X_train)
test_y_norm = output_scaler.transform(y_test)
test_x_norm = input_scaler.transform(X_test)

In [ ]:
def create_dataset (X, y, time_steps = 1):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        v = X[i:i+time_steps, :]
        Xs.append(v)
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)
TIME_STEPS = 1
X_test, y_test = create_dataset(test_x_norm, test_y_norm)
X_train, y_train = create_dataset(train_x_norm, train_y_norm)
print('X_train.shape: ', X_test.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_train.shape)

In [ ]:
def create_model_bilstm(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units = units,return_sequences=True),input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Bidirectional(LSTM(units = units)))
    model.add(Dense(1))
    #Compile model
    model.compile(loss='mse', optimizer='adam')
    return model
model_bilstm = create_model_bilstm(64)


In [ ]:
def fit_model(model):
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 10)
    history = model.fit(X_train, y_train, epochs = 100,validation_split = 0.2, batch_size = 32,shuffle = False, callbacks = [early_stop])
    return history
history_bilstm = fit_model(model_bilstm)

In [ ]:
def plot_loss (history):
    plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')
plot_loss (history_bilstm)

In [ ]:
y_test = scaler_y.inverse_transform(y_test)
y_train = scaler_y.inverse_transform(y_train)

In [ ]:
def prediction(model):
    prediction = model.predict(X_test)
    prediction = scaler_y.inverse_transform(prediction)
    return prediction
prediction_bilstm = prediction(model_bilstm)

In [ ]:
def plot_future(prediction, y_test):
    plt.figure(figsize=(10, 6))
    range_future = len(prediction)
    plt.plot(np.arange(range_future), np.array(y_test), 
             label='True Temperature')     
    plt.plot(np.arange(range_future),np.array(prediction),
            label='Prediction')
    plt.legend(loc='upper left')
    plt.xlabel('Day')
    plt.ylabel('Temperature')
plot_future(prediction_bilstm, y_test)

In [ ]:
def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print(model_name + ':')
    print('Mean Absolute Error: {:.4f}'.format(mae))
    print('Root Mean Square Error: {:.4f}'.format(rmse))

In [ ]:
evaluate_prediction(prediction_lstm, y_test, 'BILSTM')

In [ ]:
residuals=y_test-prediction_bilstm

In [ ]:

plt.scatter(residuals,prediction_bilstm)

plt.show()

In [ ]:
residuals.plot()

In [ ]:
residuals = df(residuals)
# plot residuals
residuals.plot()
pyplot.show()

In [ ]:
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
residuals = DataFrame(residuals)
# plot residuals
residuals.plot()
plt.xlabel="Day"
plt.show()

In [ ]:

plt.scatter(y_test,prediction_bilstm)
plt.xlabel("Y_test")
plt.ylabel("predicted")

plt.show()

In [ ]:
!pip install gradio

In [ ]:
def create_datasetf (X, time_steps = 1):
    Xs = []
    for i in range(1):
        v = X[i:i+time_steps, :]
        Xs.append(v)
    return np.array(Xs)

In [ ]:
def predictionn(dewptm,humidity,winddirection,visibility,windspeed,rain):
    arr = np.array([dewptm,humidity,winddirection,visibility,windspeed,rain])
    arr=arr.reshape(1,-1)
    arr = input_scaler.transform(arr)
    TIME_STEPS = 1
    arr_test = create_datasetf(arr, TIME_STEPS)
    predictionn = model_bilstm.predict(arr_test)
    predictionn = scaler_y.inverse_transform(predictionn)
    return predictionn

In [ ]:
import gradio as gr
demo = gr.Interface(
    fn=predictionn,
    inputs=["number", "number", "number","number","number",gr.Slider(0, 1)],
    outputs=["number"],
    title="Bangladesh Bi-LSTM Temperature Prediction"
)
demo.launch(debug=True)